In [17]:
import json
from dataclasses import dataclass
import numpy as np
import os
import re
from http import cookiejar
import cloudscraper
import sys
import yaml
import argparse
import glob
import logging


In [2]:
##Duration

# from dataclasses import dataclass


@dataclass(frozen=True)
class Duration:
    duration: str

    def to_seconds(self):
        if self._has_hours():
            hours, minutes, seconds = self._tokenize()
            return self._to_seconds(seconds, minutes, hours)
        elif self._has_minutes():
            minutes, seconds = self._tokenize()
            return self._to_seconds(seconds, minutes)
        elif self._has_seconds():
            [seconds] = self._tokenize()
            return self._to_seconds(seconds)
        else:
            raise ValueError("Unknown duration %s, expected format HH:MM:SS" % self.duration)

    def _tokenize(self):
        return self.duration.split(":")

    def _has_seconds(self):
        return len(self._tokenize()) == 1

    def _has_minutes(self):
        return len(self._tokenize()) == 2

    def _has_hours(self):
        return len(self._tokenize()) == 3

    @staticmethod
    def _to_seconds(seconds, minutes=0, hours=0):
        if not 0 <= int(seconds) < 60:
            raise ValueError("Seconds must be between 0 and 59 but was %s" % seconds)
        if not 0 <= int(minutes) < 60:
            raise ValueError("Minutes must be between 0 and 59 but was %s" % seconds)
        if not 0 <= int(hours) < 24:
            raise ValueError("Hours must be between 0 and 23 but was %s" % seconds)

        return int(hours) * 3600 + int(minutes) * 60 + int(seconds)


In [3]:
##Power

# from dataclasses import dataclass


@dataclass(frozen=True)
class Power:
    power: str

    def to_watts(self, ftp, diff=0):
        if not 0 <= int(ftp) < 1000:
            raise ValueError("FTP must be between 0 [W] and 999 [W] but was %s" % ftp)

        if not -1.0 < float(diff) < 1.0:
            raise ValueError("Power diff must be between -0.99 and 0.99 but was %s" % diff)

        if self._has_watt():
            absolute_power = int(self.power[:-1])
        elif self._has_percent():
            absolute_power = self._to_absolute(self.power[:-1], ftp)
        else:
            absolute_power = self._to_absolute(self.power, ftp)

        if not 0 <= int(absolute_power) < 5000:
            raise ValueError("Power must be between 0 [W] and 49999 [W] but was %s" % absolute_power)

        return round(absolute_power * (1 + diff))

    def _has_watt(self):
        return self.power.lower().endswith("w")

    def _has_percent(self):
        return self.power.lower().endswith("%")

    @staticmethod
    def _to_absolute(power, ftp):
        return int(power) * ftp / 100

In [5]:
## Functional

# import numpy as np


def flatten(xs):
    if not xs:
        return xs
    if isinstance(xs[0], list):
        return flatten(xs[0]) + flatten(xs[1:])
    return xs[:1] + flatten(xs[1:])


def fill(x, n):
    return np.full(n, x)


def filter_empty(value):
    if isinstance(value, list):
        return [filter_empty(val) for val in value if not (val is None or val == [] or val == {})]
    elif isinstance(value, dict):
        return {
            key: filter_empty(val)
            for key, val in value.items()
            if not (val is None or val == [] or val == {})
        }
    else:
        return value


def concatenate(x, y):
    return np.concatenate((x, y))

In [6]:
#Math
# import numpy as np


def moving_average(x, n):
    return np.convolve(x, np.ones((n,)) / n, mode="valid")


def normalized_power(x):
    return np.sqrt(np.sqrt(np.mean(moving_average(x, 30) ** 4)))


def intensity_factor(norm_pwr, ftp):
    return norm_pwr / ftp


def training_stress_score(seconds, norm_pwr, ftp):
    int_fct = intensity_factor(norm_pwr, ftp)
    return (seconds * norm_pwr * int_fct) / (ftp * 3600) * 100

In [7]:
##Workout

# import json

# from garminworkouts.models.duration import Duration
# from garminworkouts.models.power import Power
# from garminworkouts.utils import functional, math


class Workout(object):
    _WORKOUT_ID_FIELD = "workoutId"
    _WORKOUT_NAME_FIELD = "workoutName"
    _WORKOUT_DESCRIPTION_FIELD = "description"
    _WORKOUT_OWNER_ID_FIELD = "ownerId"

    _CYCLING_SPORT_TYPE = {
        "sportTypeId": 2,
        "sportTypeKey": "cycling"
    }

    _INTERVAL_STEP_TYPE = {
        "stepTypeId": 3,
        "stepTypeKey": "interval",
    }

    _REPEAT_STEP_TYPE = {
        "stepTypeId": 6,
        "stepTypeKey": "repeat",
    }

    def __init__(self, config, ftp, power_target_diff):
        self.config = config
        self.ftp = ftp
        self.power_target_diff = power_target_diff

    def create_workout(self, workout_id=None, workout_owner_id=None):
        return {
            self._WORKOUT_ID_FIELD: workout_id,
            self._WORKOUT_OWNER_ID_FIELD: workout_owner_id,
            self._WORKOUT_NAME_FIELD: self.get_workout_name(),
            self._WORKOUT_DESCRIPTION_FIELD: self._generate_description(),
            "sportType": self._CYCLING_SPORT_TYPE,
            "workoutSegments": [
                {
                    "segmentOrder": 1,
                    "sportType": self._CYCLING_SPORT_TYPE,
                    "workoutSteps": self._steps(self.config["steps"])
                }
            ]
        }

    def get_workout_name(self):
        return self.config["name"]

    @staticmethod
    def extract_workout_id(workout):
        return workout[Workout._WORKOUT_ID_FIELD]

    @staticmethod
    def extract_workout_name(workout):
        return workout[Workout._WORKOUT_NAME_FIELD]

    @staticmethod
    def extract_workout_description(workout):
        return workout[Workout._WORKOUT_DESCRIPTION_FIELD]

    @staticmethod
    def extract_workout_owner_id(workout):
        return workout[Workout._WORKOUT_OWNER_ID_FIELD]

    @staticmethod
    def print_workout_json(workout):
        print(json.dumps(functional.filter_empty(workout)))

    @staticmethod
    def print_workout_summary(workout):
        workout_id = Workout.extract_workout_id(workout)
        workout_name = Workout.extract_workout_name(workout)
        workout_description = Workout.extract_workout_description(workout)
        print("{0} {1:20} {2}".format(workout_id, workout_name, workout_description))

    def _generate_description(self):
        # TODO: calculate Time in Zones
        flatten_steps = functional.flatten(self.config["steps"])

        seconds = 0
        xs = []

        for step in flatten_steps:
            power = self._get_power(step)
            power_watts = power.to_watts(self.ftp) if power else None
            duration = self._get_duration(step)
            duration_secs = duration.to_seconds() if duration else None

            if power_watts and duration_secs:
                seconds = seconds + duration_secs
                xs = functional.concatenate(xs, functional.fill(power_watts, duration_secs))

        norm_pwr = math.normalized_power(xs)
        int_fct = math.intensity_factor(norm_pwr, self.ftp)
        tss = math.training_stress_score(seconds, norm_pwr, self.ftp)

        return "FTP %d, TSS %d, NP %d, IF %.2f" % (self.ftp, tss, norm_pwr, int_fct)

    def _steps(self, steps_config):
        steps, step_order, child_step_id = self._steps_recursive(steps_config, 0, None)
        return steps

    def _steps_recursive(self, steps_config, step_order, child_step_id):
        if not steps_config:
            return [], step_order, child_step_id

        steps_config_agg = [(1, steps_config[0])]

        for step_config in steps_config[1:]:
            (repeats, prev_step_config) = steps_config_agg[-1]
            if prev_step_config == step_config:  # repeated step
                steps_config_agg[-1] = (repeats + 1, step_config)
            else:
                steps_config_agg.append((1, step_config))

        steps = []
        for repeats, step_config in steps_config_agg:
            step_order = step_order + 1
            if isinstance(step_config, list):
                child_step_id = child_step_id + 1 if child_step_id else 1

                repeat_step_order = step_order
                repeat_child_step_id = child_step_id

                nested_steps, step_order, child_step_id = self._steps_recursive(step_config, step_order, child_step_id)
                steps.append(self._repeat_step(repeat_step_order, repeat_child_step_id, repeats, nested_steps))
            else:
                steps.append(self._interval_step(step_config, child_step_id, step_order))

        return steps, step_order, child_step_id

    def _repeat_step(self, step_order, child_step_id, repeats, nested_steps):
        return {
            "type": "RepeatGroupDTO",
            "stepOrder": step_order,
            "stepType": self._REPEAT_STEP_TYPE,
            "childStepId": child_step_id,
            "numberOfIterations": repeats,
            "workoutSteps": nested_steps,
            "smartRepeat": False
        }

    def _interval_step(self, step_config, child_step_id, step_order):
        return {
            "type": "ExecutableStepDTO",
            "stepOrder": step_order,
            "stepType": self._INTERVAL_STEP_TYPE,
            "childStepId": child_step_id,
            "endCondition": self._end_condition(step_config),
            "endConditionValue": self._end_condition_value(step_config),
            "targetType": self._target_type(step_config),
            "targetValueOne": self._target_value_one(step_config),
            "targetValueTwo": self._target_value_two(step_config)
        }

    @staticmethod
    def _get_duration(step_config):
        duration = step_config.get("duration")
        return Duration(str(duration)) if duration else None

    def _end_condition(self, step_config):
        duration = self._get_duration(step_config)
        type_id = 2 if duration else 1
        type_key = "time" if duration else "lap.button"
        return {
            "conditionTypeId": type_id,
            "conditionTypeKey": type_key
        }

    def _end_condition_value(self, step_config):
        duration = self._get_duration(step_config)
        return duration.to_seconds() if duration else None

    @staticmethod
    def _get_power(step):
        power = step.get("power")
        return Power(str(power)) if power else None

    def _target_type(self, step_config):
        power = self._get_power(step_config)
        type_id = 2 if power else 1
        type_key = "power.zone" if power else "no.target"
        return {
            "workoutTargetTypeId": type_id,
            "workoutTargetTypeKey": type_key
        }

    def _target_value_one(self, step_config):
        power = self._get_power(step_config)
        return power.to_watts(self.ftp, -self.power_target_diff) if power else None

    def _target_value_two(self, step_config):
        power = self._get_power(step_config)
        return power.to_watts(self.ftp, +self.power_target_diff) if power else None


In [9]:
##session for connect and disconnect

# import os
# import re
# from http import cookiejar

# import cloudscraper


def connect(connect_url, sso_url, username, password, cookie_jar):
    session = cloudscraper.CloudScraper()
    _load_cookie_jar(session, cookie_jar)

    url = connect_url + "/modern/settings"
    response = session.get(url, allow_redirects=False)
    if response.status_code != 200:
        _authenticate(session, connect_url, sso_url, username, password)

    return session


def disconnect(session):
    _save_cookie_jar(session)
    session.close()


def _load_cookie_jar(session, cookie_jar):
    if cookie_jar:
        session.cookies = cookiejar.LWPCookieJar(cookie_jar)
        if os.path.isfile(cookie_jar):
            session.cookies.load(ignore_discard=True, ignore_expires=True)


def _save_cookie_jar(session):
    if isinstance(session.cookies, cookiejar.LWPCookieJar):
        session.cookies.save(ignore_discard=True, ignore_expires=True)


def _authenticate(session, connect_url, sso_url, username, password):
    url = sso_url + "/sso/signin"
    headers = {'origin': 'https://sso.garmin.com'}
    params = {
        "service": "https://connect.garmin.com/modern"
    }
    data = {
        "username": username,
        "password": password,
        "embed": "false"
    }

    auth_response = session.post(url, headers=headers, params=params, data=data)
    auth_response.raise_for_status()

    auth_ticket = _extract_auth_ticket(auth_response.text)

    response = session.get(connect_url + "/modern", params={"ticket": auth_ticket})
    response.raise_for_status()


def _extract_auth_ticket(auth_response):
    match = re.search(r'response_url\s*=\s*".*\?ticket=(.+)"', auth_response)
    if not match:
        raise Exception("Unable to extract auth ticket URL from:\n%s" % auth_response)
    return match.group(1)


In [11]:
## GarminClient

# import json
# import sys

# from garminworkouts.garmin.session import connect, disconnect


class GarminClient(object):
    _WORKOUT_SERVICE_ENDPOINT = "/proxy/workout-service"

    _REQUIRED_HEADERS = {
        "Referer": "https://connect.garmin.com/modern/workouts",
        "nk": "NT"
    }

    def __init__(self, connect_url, sso_url, username, password, cookie_jar):
        self.connect_url = connect_url
        self.sso_url = sso_url
        self.username = username
        self.password = password
        self.cookie_jar = cookie_jar

    def __enter__(self):
        self.session = connect(self.connect_url, self.sso_url, self.username, self.password, self.cookie_jar)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        disconnect(self.session)

    def list_workouts(self, batch_size=100):
        for start_index in range(0, sys.maxsize, batch_size):

            url = f"{self.connect_url}{GarminClient._WORKOUT_SERVICE_ENDPOINT}/workouts"
            params = {
                "start": start_index,
                "limit": batch_size
            }
            response = self.session.get(url, headers=GarminClient._REQUIRED_HEADERS, params=params)
            response.raise_for_status()

            response_jsons = json.loads(response.text)
            if not response_jsons or response_jsons == []:
                break

            for response_json in response_jsons:
                yield response_json

    def get_workout(self, workout_id):
        url = f"{self.connect_url}{GarminClient._WORKOUT_SERVICE_ENDPOINT}/workout/{workout_id}"

        response = self.session.get(url, headers=GarminClient._REQUIRED_HEADERS)
        response.raise_for_status()

        return json.loads(response.text)

    def download_workout(self, workout_id, file):
        url = f"{self.connect_url}{GarminClient._WORKOUT_SERVICE_ENDPOINT}/workout/FIT/{workout_id}"

        response = self.session.get(url)
        response.raise_for_status()

        with open(file, "wb") as f:
            f.write(response.content)

    def save_workout(self, workout):
        url = f"{self.connect_url}{GarminClient._WORKOUT_SERVICE_ENDPOINT}/workout"

        response = self.session.post(url, headers=GarminClient._REQUIRED_HEADERS, json=workout)
        response.raise_for_status()

    def update_workout(self, workout_id, workout):
        url = f"{self.connect_url}{GarminClient._WORKOUT_SERVICE_ENDPOINT}/workout/{workout_id}"

        response = self.session.put(url, headers=GarminClient._REQUIRED_HEADERS, json=workout)
        response.raise_for_status()

    def delete_workout(self, workout_id):
        url = f"{self.connect_url}{GarminClient._WORKOUT_SERVICE_ENDPOINT}/workout/{workout_id}"

        response = self.session.delete(url, headers=GarminClient._REQUIRED_HEADERS)
        response.raise_for_status()

    def schedule_workout(self, workout_id, date):
        url = f"{self.connect_url}{GarminClient._WORKOUT_SERVICE_ENDPOINT}/schedule/{workout_id}"
        json_data = {"date": date}

        response = self.session.post(url, headers=GarminClient._REQUIRED_HEADERS, json=json_data)
        response.raise_for_status()


In [12]:
##Excel to yaml

# import numpy as np
# import pandas as pd


def excel_to_yaml(filename, seconds_block=10):  # noqa: C901
    df = pd.read_excel(filename)
    df = df[df.columns.tolist()[:3]]
    df.columns = ["start", "end", "duration"]
    df.reset_index(inplace=True, drop=True)

    def normalise(x):
        if str(x) == str(np.nan):
            return x
        elif "W" in str(x):
            x = str(x).replace("W", "").replace(" ", "").replace(",", ".")
            x = float(x)
            return x
        else:
            x = float(x)
            return x

    def check_power_type(x):
        if "W" in str(x):
            return "W"
        else:
            return ""

    df["power_type"] = df["start"].apply(lambda x: check_power_type(x))

    df["start"] = df["start"].apply(lambda x: normalise(x))
    df["end"] = df["end"].apply(lambda x: normalise(x))
    df["duration"] = df["duration"].astype(str)

    def check_n_steps(df, seconds_block=seconds_block):
        if str(df["end"]) == str(np.nan):
            return 1
        elif df["start"] == df["end"]:
            return 1
        elif df["start"] != df["end"]:
            duration = df["duration"]
            seconds = int(duration.split(":")[-1]) + int(duration.split(":")[-2]) * 60
            n_blocks = seconds / seconds_block
            n_blocks = round(n_blocks, 0)
            return n_blocks

    df["n_blocks"] = df.apply(check_n_steps, axis=1)

    total_steps = df["n_blocks"].sum()

    if total_steps > 50:
        one_steps = len(total_steps[total_steps["n_blocks"] == 1])
        more_than_one_step = total_steps - one_steps

        # one_steps + more_than_one_steps <= 50

        max_more_than_one = 50 - one_steps
        perc = max_more_than_one / more_than_one_step

        df["n_blocks"] = df["n_blocks"].apply(lambda x: int(x * perc))

    workout_name = filename.replace("_", " ").split(".xls")[0].split("/")[-1]
    workout_name = 'name: "{}"\n'.format(workout_name)

    def create_steps(i, df=df):
        start = df.loc[i, "start"]
        end = df.loc[i, "end"]
        duration = df.loc[i, "duration"]
        n_blocks = df.loc[i, "n_blocks"]
        power_type = df.loc[i, "power_type"]
        if n_blocks == 1:
            template = '  - {{ power: {power}{power_type}, duration: "{duration}" }}\n'
            return template.format(power=int(start), power_type=power_type, duration=duration)
        elif n_blocks > 1:
            seconds = int(duration.split(":")[-1]) + int(duration.split(":")[-2]) * 60
            # n_blocks=seconds/seconds_block
            seconds_block = int(seconds / n_blocks)
            minutes_ = int(seconds_block / 60)
            seconds_ = int((float(seconds_block / 60) - int(seconds_block / 60)) * 60)
            duration = "{minutes}:{seconds}".format(minutes=minutes_, seconds=seconds_)
            power_dif = abs(end - start)
            power_step = power_dif / n_blocks
            steps_text = ""
            total_time = 0
            while total_time <= seconds:
                text = '  - {{ power: {0}{1}, duration: "{2}" }}\n'.format(int(start), power_type, duration)
                steps_text = steps_text + text
                if start <= end:
                    start = start + power_step
                elif start >= end:
                    start = start - power_step
                total_time = total_time + seconds_block
            return steps_text

    steps = "steps:\n"
    for i in df.index.tolist():
        step = create_steps(i)
        steps = steps + step

    yaml_text = '{workout_name}\n{steps}'.format(workout_name=workout_name, steps=steps)

    filename = filename.split(".xls")[0] + ".yaml"
    with open(filename, "w") as fout:
        fout.write(yaml_text)
    return filename


In [15]:
##IncludeLoader

# import os

# import yaml


class IncludeLoader(yaml.SafeLoader):

    def __init__(self, stream):
        self._root = os.path.split(stream.name)[0]

        super(IncludeLoader, self).__init__(stream)

    def include(self, node):
        filename = os.path.join(self._root, self.construct_scalar(node))

        with open(filename, 'r') as f:
            return yaml.load(f, IncludeLoader)


IncludeLoader.add_constructor('!include', IncludeLoader.include)

In [16]:
##configreader

# import yaml

# from garminworkouts.config.excelparser import excel_to_yaml
# from garminworkouts.config.includeloader import IncludeLoader


def read_config(filename):
    if "xls" in filename.split(".")[-1]:
        filename = excel_to_yaml(filename)

    with open(filename, 'r') as f:
        data = yaml.load(f, IncludeLoader)
    return data


In [18]:
##envdefault

# import argparse
# import os


class EnvDefault(argparse.Action):
    def __init__(self, env_var, required=True, default=None, **kwargs):
        if not default and env_var:
            if env_var in os.environ:
                default = os.environ[env_var]
        if required and default:
            required = False
        super(EnvDefault, self).__init__(default=default, required=required,
                                         **kwargs)

    def __call__(self, parser, namespace, values, option_string=None):
        setattr(namespace, self.dest, values)

In [19]:
##validators

import argparse
import os


def writeable_dir(directory):
    if not os.path.isdir(directory):
        raise argparse.ArgumentTypeError("'%s' is not a directory" % directory)
    if not os.access(directory, os.W_OK):
        raise argparse.ArgumentTypeError("'%s' is not a writeable directory" % directory)
    return directory


In [20]:
##main

#!/usr/bin/env python3

# import argparse
# import glob
# import logging
# import os

# from garminworkouts.config import configreader
# from garminworkouts.garmin.garminclient import GarminClient
# from garminworkouts.models.workout import Workout
# from garminworkouts.utils.validators import writeable_dir
# from garminworkouts.utils.envdefault import EnvDefault


def command_import(args):
    workout_files = glob.glob(args.workout)

    workout_configs = [configreader.read_config(workout_file) for workout_file in workout_files]
    workouts = [Workout(workout_config, args.ftp, args.target_power_diff) for workout_config in workout_configs]

    with _garmin_client(args) as connection:
        existing_workouts_by_name = {Workout.extract_workout_name(w): w for w in connection.list_workouts()}

        for workout in workouts:
            workout_name = workout.get_workout_name()
            existing_workout = existing_workouts_by_name.get(workout_name)

            if existing_workout:
                workout_id = Workout.extract_workout_id(existing_workout)
                workout_owner_id = Workout.extract_workout_owner_id(existing_workout)
                payload = workout.create_workout(workout_id, workout_owner_id)
                logging.info("Updating workout '%s'", workout_name)
                connection.update_workout(workout_id, payload)
            else:
                payload = workout.create_workout()
                logging.info("Creating workout '%s'", workout_name)
                connection.save_workout(payload)


def command_export(args):
    with _garmin_client(args) as connection:
        for workout in connection.list_workouts():
            workout_id = Workout.extract_workout_id(workout)
            workout_name = Workout.extract_workout_name(workout)
            file = os.path.join(args.directory, str(workout_id)) + ".fit"
            logging.info("Exporting workout '%s' into '%s'", workout_name, file)
            connection.download_workout(workout_id, file)


def command_list(args):
    with _garmin_client(args) as connection:
        for workout in connection.list_workouts():
            Workout.print_workout_summary(workout)


def command_schedule(args):
    with _garmin_client(args) as connection:
        workout_id = args.workout_id
        date = args.date
        connection.schedule_workout(workout_id, date)


def command_get(args):
    with _garmin_client(args) as connection:
        workout = connection.get_workout(args.id)
        Workout.print_workout_json(workout)


def command_delete(args):
    with _garmin_client(args) as connection:
        logging.info("Deleting workout '%s'", args.id)
        connection.delete_workout(args.id)


def _garmin_client(args):
    return GarminClient(
        connect_url=args.connect_url,
        sso_url=args.sso_url,
        username=args.username,
        password=args.password,
        cookie_jar=args.cookie_jar
    )


def main():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter,
                                     description="Manage Garmin Connect workout(s)")
    parser.add_argument("--username", "-u", action=EnvDefault, env_var="GARMIN_USERNAME",
                        required=True, help="Garmin Connect account username")
    parser.add_argument("--password", "-p", action=EnvDefault, env_var="GARMIN_PASSWORD",
                        required=True, help="Garmin Connect account password")
    parser.add_argument("--cookie-jar", default=".garmin-cookies.txt", help="Filename with authentication cookies")
    parser.add_argument("--connect-url", default="https://connect.garmin.com", help="Garmin Connect url")
    parser.add_argument("--sso-url", default="https://sso.garmin.com", help="Garmin SSO url")
    parser.add_argument("--debug", action='store_true', help="Enables more detailed messages")

    subparsers = parser.add_subparsers(title="Commands")

    parser_import = subparsers.add_parser("import", description="Import workout(s) from file(s) into Garmin Connect")
    parser_import.add_argument("workout",
                               help="File(s) with workout(s) to import, "
                                    "wildcards are supported e.g: sample_workouts/*.yaml")
    parser_import.add_argument("--ftp", required=True, type=int,
                               help="FTP to calculate absolute target power from relative value")
    parser_import.add_argument("--target-power-diff", default=0.05, type=float,
                               help="Percent of target power to calculate final target power range")
    parser_import.set_defaults(func=command_import)

    parser_export = subparsers.add_parser("export",
                                          description="Export all workouts from Garmin Connect and save into directory")
    parser_export.add_argument("directory", type=writeable_dir,
                               help="Destination directory where workout(s) will be exported")
    parser_export.set_defaults(func=command_export)

    parser_list = subparsers.add_parser("list", description="List all workouts")
    parser_list.set_defaults(func=command_list)

    parser_schedule = subparsers.add_parser("schedule", description="Schedule a workouts")
    parser_schedule.add_argument("--workout_id", "-w", required=True, help="Workout id to schedule")
    parser_schedule.add_argument("--date", "-d", required=True, help="Date to which schedule the workout")
    parser_schedule.set_defaults(func=command_schedule)

    parser_get = subparsers.add_parser("get", description="Get workout")
    parser_get.add_argument("--id", required=True, help="Workout id, use list command to get workouts identifiers")
    parser_get.set_defaults(func=command_get)

    parser_delete = subparsers.add_parser("delete", description="Delete workout")
    parser_delete.add_argument("--id", required=True, help="Workout id, use list command to get workouts identifiers")
    parser_delete.set_defaults(func=command_delete)

    args = parser.parse_args()

    logging_level = logging.DEBUG if args.debug else logging.INFO
    logging.basicConfig(level=logging_level)

    args.func(args)


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] --username USERNAME --password PASSWORD
                             [--cookie-jar COOKIE_JAR]
                             [--connect-url CONNECT_URL] [--sso-url SSO_URL]
                             [--debug]
                             {import,export,list,schedule,get,delete} ...
ipykernel_launcher.py: error: the following arguments are required: --username/-u, --password/-p


SystemExit: 2

/Users/rick_clayton/Dev/garmin-workouts/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
